In [ ]:
'''
Final Project:
Implementing a Convolutional Neural Network based on reviewed papers
For the Task of Age Estimation
Based on the following paper:
C. Miron, V. Manta, R. Timofte, A. Pasarica, R. Ciucu, “Efficient convolutional neural network for
apparent age prediction”, 2019 IEEE 15th International Conference on Intelligent Computer
Communication and Processing (ICCP)

Datasets:
    UTKFace available at https://susanqq.github.io/UTKFace/
    APPA-REAL available at http://158.109.8.102/AppaRealAge/appa-real-release.zip
Original Google Colab Notebook available at https://colab.research.google.com/drive/1GTDfkU_g1YEKDk64PTF7t9Rnzl61gct5?usp=sharing

Ali Jedari Heidarzadeh
University of Tabriz, Tabriz, Iran
Winter 2021-2022
'''

'\nFinal Project:\nImplementing a Convolutional Neural Network based on reviewed papers\nfor the task of age estimation\nDataset: UTKFace available at https://susanqq.github.io/UTKFace/\n\nAli Jedari Heidarzadeh\nUniversity of Tabriz, Tabriz, Iran\nWinter 2021-2022\n'

In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.model_selection import train_test_split

In [ ]:
def utk_data():
  path = '/content/UTKFace/'
  files = os.listdir(path)
  images, ages = [], []
  for f in files:
    ages.append(int(f.split('_')[0]))
    image = cv2.imread(path+f)
    image = cv2.resize(image, dsize=(50, 50))
    images.append(image / 255)
  images = np.array(images)
  ages = np.array(ages)
  train_samples, X_test, train_labels, y_test = train_test_split(images, ages, test_size=0.15, shuffle=True)
  X_train, X_valid, y_train, y_valid = train_test_split(train_samples, train_labels, test_size=0.05, shuffle=True)

  return X_train, X_valid, X_test, y_train, y_valid, y_test

In [ ]:
X_train, X_valid, X_test, y_train, y_valid, y_test = utk_data()

In [ ]:
def appa_real_data():
  path = '/content/appa-real-release/'
  train_path = '/content/appa-real-release/train/'
  test_path = '/content/appa-real-release/test/'
  valid_path = '/content/appa-real-release/valid/'
  
  gt = pd.read_csv(path+'gt_avg_train.csv')
  tr = gt['real_age'].copy()

  X = []
  for i in range(gt.shape[0]):
    img = cv2.imread(train_path+gt.iloc[i]['file_name']+'_face.jpg')
    X.append(cv2.resize(img, dsize=(50, 50)))    
  
  gt = pd.read_csv(path+'gt_avg_test.csv')
  ts = gt['real_age'].copy()

  for i in range(gt.shape[0]):
    img = cv2.imread(test_path+gt.iloc[i]['file_name']+'_face.jpg')
    X.append(cv2.resize(img, dsize=(50, 50)))

  gt = pd.read_csv(path+'gt_avg_valid.csv')
  vl = gt['real_age'].copy()

  for i in range(gt.shape[0]):
    img = cv2.imread(valid_path+gt.iloc[i]['file_name']+'_face.jpg')
    X.append(cv2.resize(img, dsize=(50, 50)))

  X = np.array(X) / 255
  y = pd.concat([tr, ts, vl])
  train, X_test, train_labels, y_test = train_test_split(X, y, test_size=0.15, shuffle=True)
  X_train, X_valid, y_train, y_valid = train_test_split(train, train_labels, test_size=0.1, shuffle=True)
  return X_train, X_valid, X_test, y_train, y_valid, y_test

In [ ]:
X_train, X_valid, X_test, y_train, y_valid, y_test = appa_real_data()

In [ ]:
mx = np.max(np.array([y_train.max(), y_test.max(), y_valid.max()]))
mn = np.min(np.array([y_train.min(), y_test.min(), y_valid.min()]))
output_shape = mx - mn + 1

In [ ]:
print(mx)
print(mn)
print(output_shape)

100
1
100


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()

In [ ]:
model.add(Input(shape=(50, 50, 3)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu"))
model.add(Flatten())
model.add(Dense(output_shape, activation='relu'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 46, 46, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 44, 44, 32)        9248      
                                                                 
 batch_normalization_2 (Batc  (None, 44, 44, 32)       1

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30
182/182 [==============================] - 122s 662ms/step - loss: 487.7621 - mae: 16.8544 - val_loss: 608.5001 - val_mae: 19.4362
Epoch 2/30
182/182 [==============================] - 123s 675ms/step - loss: 282.1091 - mae: 12.6764 - val_loss: 310.4557 - val_mae: 13.5779
Epoch 3/30
182/182 [==============================] - 118s 648ms/step - loss: 235.8981 - mae: 11.5725 - val_loss: 262.9206 - val_mae: 12.5867
Epoch 4/30
 95/182 [==============>...............] - ETA: 55s - loss: 206.5880 - mae: 10.6322

KeyboardInterrupt: ignored

In [ ]:
results = model.evaluate(X_test, y_test)

36/36 [==============================] - 5s 136ms/step - loss: 195.4360 - mae: 10.4663


In [ ]:
#from tensorflow import keras
#Uncomment the following line, if you want to save the model
#model.save('Age_Estimation_CNN.h5')
#Uncomment the following line, if you want to load the model
#model = keras.models.load_model('Age_Estimation_CNN.h5')

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('Error')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'])
plt.show()